<a href="https://colab.research.google.com/github/petewarden/pico_colabs/blob/main/Building_Person_Detection_for_the_Arducam_Pico4ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Person Detection for the Arducam Pico4ML

*By [Pete Warden](https://twitter.com/petewarden), peteward@stanford.edu*

This notebook demonstrates how to compile and deploy some programs for [the Arducam Pico4ML board](), which is based on a Raspberry Pi Pico RP2040 microcontroller, but adds sensors and a screen to make it easier to use for TinyML development. A lot of the setup steps are the same as for [the blink Pico example](https://colab.research.google.com/github/petewarden/pico_colabs/blob/main/Building_Blink_for_the_Raspberry_Pi_Pico.ipynb) so I don't go into as much detail on the basics.

## Install the SDK and Examples

This is similar to the blink notebook, but instead of installing the example code for the standalone Pico board, we fetch [the Arducam repository of sample code](https://github.com/ArduCAM/RPI-Pico-Cam).

In [1]:
!mkdir pico
%cd pico
!git clone -b master https://github.com/raspberrypi/pico-sdk.git
%cd pico-sdk
!git submodule update --init
%cd ..
!git clone -b master https://github.com/ArduCAM/RPI-Pico-Cam

/content/pico
Cloning into 'pico-sdk'...
remote: Enumerating objects: 5518, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 5518 (delta 19), reused 24 (delta 11), pack-reused 5476
Receiving objects: 100% (5518/5518), 2.51 MiB | 9.01 MiB/s, done.
Resolving deltas: 100% (2788/2788), done.
/content/pico/pico-sdk
Submodule 'lib/cyw43-driver' (https://github.com/georgerobotics/cyw43-driver.git) registered for path 'lib/cyw43-driver'
Submodule 'lib/lwip' (https://github.com/lwip-tcpip/lwip.git) registered for path 'lib/lwip'
Submodule 'tinyusb' (https://github.com/hathach/tinyusb.git) registered for path 'lib/tinyusb'
Cloning into '/content/pico/pico-sdk/lib/cyw43-driver'...
Cloning into '/content/pico/pico-sdk/lib/lwip'...
Cloning into '/content/pico/pico-sdk/lib/tinyusb'...
Submodule path 'lib/cyw43-driver': checked out '195dfcc10bb6f379e3dea45147590db2203d3c7b'
Submodule path 'lib/lwip': checked out '239918ccc173cb2c2a62f4

## Install the Toolchain

In [2]:
!sudo apt update
!sudo apt install cmake gcc-arm-none-eabi libnewlib-arm-none-eabi build-essential libstdc++-arm-none-eabi-newlib

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.laun

## Setting up the Build System for the Video Streaming Example

The first program we're going to build is an example that takes the output from the builtin camera on the Pico4ML board and copies it to the screen. This is simpler than the person detection example, so it's a good way to make sure we have the system working well.

Because the folder structure of the Arducam repository is a little different from the Raspberry Pi examples, we have to set the `PICO_SDK_PATH` a bit differently too.

In [3]:
%cd /content/pico/RPI-Pico-Cam/arducam_demo
!mkdir build 
%cd build 
%env PICO_SDK_PATH=/content/pico/pico-sdk
!cmake ..

/content/pico/RPI-Pico-Cam/arducam_demo
/content/pico/RPI-Pico-Cam/arducam_demo/build
env: PICO_SDK_PATH=/content/pico/pico-sdk
Using PICO_SDK_PATH from environment ('/content/pico/pico-sdk')
PICO_SDK_PATH is /content/pico/pico-sdk
Defaulting PICO_PLATFORM to rp2040 since not specified.
Defaulting PICO platform compiler to pico_arm_gcc since not specified.
-- Defaulting build type to 'Release' since not specified.
PICO compiler is pico_arm_gcc
-- The C compiler identification is GNU 6.3.1
-- The CXX compiler identification is GNU 6.3.1
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/arm-none-eabi-gcc
Build type is Release
Defaulting PICO target board to pico since not specified.
Using board configuration from /content/pico/pico-sdk/src/boards/include/boards/pico.h
-- Found Python3: /usr/bin/python3.7 (found version "3.7.14") found components: Interpreter 
TinyUSB available at /content/pico/pico-sdk/lib/tinyusb/src/portable/raspberrypi/rp2040; enabling build suppo

## Compiling the Video Streaming Example

Now we're going to build the executable for the video streaming.

In [5]:
%cd /content/pico/RPI-Pico-Cam/arducam_demo/build
!make -j4

/content/pico/RPI-Pico-Cam/arducam_demo/build
Scanning dependencies of target bs2_default
[  0%] Building ASM object pico-sdk/src/rp2_common/boot_stage2/CMakeFiles/bs2_default.dir/compile_time_choice.S.obj
[  1%] Creating directories for 'ELF2UF2Build'
[  1%] Creating directories for 'PioasmBuild'
[  2%] Linking ASM executable bs2_default.elf
[  2%] Built target bs2_default
[  2%] No download step for 'ELF2UF2Build'
[  3%] No download step for 'PioasmBuild'
[  4%] Generating bs2_default.bin
[  4%] Generating bs2_default_padded_checksummed.S
[  5%] No update step for 'ELF2UF2Build'
[  6%] No update step for 'PioasmBuild'
[  7%] No patch step for 'PioasmBuild'
[  7%] No patch step for 'ELF2UF2Build'
[  7%] Built target bs2_default_padded_checksummed_asm
Scanning dependencies of target arducam
[  8%] Performing configure step for 'PioasmBuild'
[  9%] Performing configure step for 'ELF2UF2Build'
[ 10%] Building C object Arducam/src/CMakeFiles/arducam.dir/content/pico/pico-sdk/src/rp2_commo

## Load and Run the Executable

You should now be able to find the video streaming binary in `/content/pico/RPI-Pico-Cam/arducam_demo/build/arducam/arducam_demo.uf2`. Press the `bootsel` button on the board, plug it into the USB socket so that the mass storage drive appears in the file system, and copy the binary over. After the board reboots, you should see the output of the camera in the board's display.